# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, ml_methods_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils
from null_behaviors import find_best_arc, curvature_utils, curv_of_traj_utils, opt_arc_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_cv_utils_stage2 import cca_cv_utils2, cca_cv_compare_data
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import pn_feature_selection

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# Retrieve data

## get data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0402"

In [ ]:
reload(opt_arc_utils)

In [ ]:
reduce_y_var_lags = True
planning_data_by_point_exists_ok = True
y_data_exists_ok = True

pn = planning_and_neural_class.PlanningAndNeural(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
    pn.planning_data_by_point)
pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

In [ ]:
stop!

# CV: one dataset

In [ ]:
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_cv_utils_stage2 import cca_cv_utils2, cca_cv_compare_data, pinpoint_features

X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn_feature_selection.select_features(pn.y_var).copy()

cv_summary = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2, trial_ids=pn.y_var['segment'].values)
cca_cv_utils2.plot_cv_cca_overlay(cv_summary, show_train=True)


In [ ]:
cca_cv_utils2.plot_cv_cca_violin(cv_summary)

In [ ]:
cca_cv_utils2.plot_cv_cca_lines(cv_summary)

# Selected features vs rest

In [ ]:
X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn_feature_selection.select_features(pn.y_var).copy()
X2_rest = pn.y_var[[col for col in pn.y_var.columns if col not in X2.columns]].copy()

cv_summary = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2, trial_ids=pn.y_var['segment'].values)
cv_summary2 = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2_rest, trial_ids=pn.y_var['segment'].values)

label1 = "Selected Features"
label2 = "Rest"
cca_cv_compare_data.plot_cv_cca_compare_all(cv_summary, cv_summary2, labels=(label1, label2))
# # Or call individual plots if you prefer:
# cca_cv_compare_data.plot_cv_means_sd(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_all_folds_overlay(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_mean_difference(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_train_test_gap(cv_summary, cv_summary2, labels=(label1, label2))


# Eye features vs rest


In [ ]:
# Define keywords once
eye_keywords = ("eye", "gaze", "LD", "RD")
eye_cols = [col for col in pn.y_var.columns if any(k in col for k in eye_keywords)]


X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn.y_var[eye_cols].copy()

cca_no_lag = cca_class.CCAclass(X1=X1, 
                                X2=X2, 
                                lagging_included=False)
cca_no_lag.conduct_cca()

cca_rest = cca_class.CCAclass(X1=X1, 

In [ ]:
eye_keywords = ("eye", "gaze", "LD", "RD")
eye_cols = [col for col in pn.y_var.columns if any(k in col for k in eye_keywords)]

X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn.y_var[eye_cols].copy()
X2_rest = pn.y_var[[col for col in pn.y_var.columns if col not in X2.columns]].copy()

cv_summary = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2, trial_ids=pn.y_var['segment'].values)
cv_summary2 = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2_rest, trial_ids=pn.y_var['segment'].values)

label1 = "Selected Features"
label2 = "Rest"
cca_cv_compare_data.plot_cv_cca_compare_all(cv_summary, cv_summary2, labels=(label1, label2))

# Selected features vs Eyes

In [ ]:
X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()

eye_keywords = ("eye", "gaze", "LD", "RD")
eye_cols = [col for col in pn.y_var.columns if any(k in col for k in eye_keywords)]
X2 = pn_feature_selection.select_features(pn.y_var).copy()

X2_rest = pd.concat([X2, pn.y_var[eye_cols]], axis=1)

cv_summary = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2, trial_ids=pn.y_var['segment'].values)
cv_summary2 = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2_rest, trial_ids=pn.y_var['segment'].values)


label1 = "Selected Features"
label2 = "Selected Features + Eyes"
cca_cv_compare_data.plot_cv_cca_compare_all(cv_summary, cv_summary2, labels=(label1, label2))
# # Or call individual plots if you prefer:
# cca_cv_compare_data.plot_cv_means_sd(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_all_folds_overlay(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_mean_difference(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_train_test_gap(cv_summary, cv_summary2, labels=(label1, label2))


# Selected features vs All

In [ ]:
X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn_feature_selection.select_features(pn.y_var).copy()

X2_rest = pn.y_var.copy()

cv_summary = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2, trial_ids=pn.y_var['segment'].values)
cv_summary2 = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2_rest, trial_ids=pn.y_var['segment'].values)


label1 = "Selected Features"
label2 = "All"
cca_cv_compare_data.plot_cv_cca_compare_all(cv_summary, cv_summary2, labels=(label1, label2))
# # Or call individual plots if you prefer:
# cca_cv_compare_data.plot_cv_means_sd(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_all_folds_overlay(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_mean_difference(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_train_test_gap(cv_summary, cv_summary2, labels=(label1, label2))


# Selected features + Eyes vs All Reduced

In [ ]:
X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()

eye_keywords = ("eye", "gaze", "LD", "RD")
eye_cols = [col for col in pn.y_var.columns if any(k in col for k in eye_keywords)]
X2 = pn_feature_selection.select_features(pn.y_var).copy()
X2 = pd.concat([X2, pn.y_var[eye_cols]], axis=1)

X2_rest = pn.y_var_reduced.copy()

cv_summary = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2, trial_ids=pn.y_var['segment'].values)
cv_summary2 = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2_rest, trial_ids=pn.y_var['segment'].values)


label1 = "Selected Features + Eyes"
label2 = "All features reduced"
cca_cv_compare_data.plot_cv_cca_compare_all(cv_summary, cv_summary2, labels=(label1, label2))
# # Or call individual plots if you prefer:
# cca_cv_compare_data.plot_cv_means_sd(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_all_folds_overlay(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_mean_difference(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_train_test_gap(cv_summary, cv_summary2, labels=(label1, label2))


# All vs Reduced

In [ ]:
X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn.y_var.copy()
X2_rest = pn.y_var_reduced.copy()

cv_summary = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2, trial_ids=pn.y_var['segment'].values)
cv_summary2 = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2_rest, trial_ids=pn.y_var['segment'].values)


label1 = "All features"
label2 = "All features reduced"
cca_cv_compare_data.plot_cv_cca_compare_all(cv_summary, cv_summary2, labels=(label1, label2))
# # Or call individual plots if you prefer:
# cca_cv_compare_data.plot_cv_means_sd(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_all_folds_overlay(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_mean_difference(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_train_test_gap(cv_summary, cv_summary2, labels=(label1, label2))


In [ ]:
X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn.y_var.copy()
X2_rest = pn.y_var_reduced.copy()

cv_summary = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2, trial_ids=pn.y_var['segment'].values)
cv_summary2 = cca_cv_utils2.conduct_cca_cv(X1_unscaled=X1, X2_unscaled=X2_rest, trial_ids=pn.y_var['segment'].values)


label1 = "All features"
label2 = "All features reduced"
cca_cv_compare_data.plot_cv_cca_compare_all(cv_summary, cv_summary2, labels=(label1, label2))
# # Or call individual plots if you prefer:
# cca_cv_compare_data.plot_cv_means_sd(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_all_folds_overlay(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_mean_difference(cv_summary, cv_summary2, labels=(label1, label2))
# cca_cv_compare_data.plot_cv_train_test_gap(cv_summary, cv_summary2, labels=(label1, label2))


# Pinpoint features

In [ ]:
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_cv_utils_stage2 import cca_cv_utils2, cca_cv_compare_data, pinpoint_features

## CV permutation importance (no retraining per feature)

In [ ]:
X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
# X2 = pn.y_var.drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn.y_var.copy()

scaler = StandardScaler()
X1_sc, X2_sc = scaler.fit_transform(
    X1), scaler.fit_transform(X2)

# all_names is a list of feature names for the all data set
imp = pinpoint_features.cv_cca_perm_importance(X2_sc, X1_sc, X2.columns,
                             n_components=10, reg=1e-2, n_splits=5,
                             groups=pn.y_var['segment'].values, max_components_for_score=3)
imp.head(20)  # top contributors


## CV leave-one-out Δρ (re-train per feature)

In [ ]:
# Leave-one-out Δρ for all features (re-fit per feature)
imp_loo = cv_cca_leave1out_delta(
    X2_sc, X1_sc, X2.columns,
    n_components=10, reg=1e-2,
    n_splits=5, random_state=0, groups=pn.y_var['segment'].values,
    max_components_for_score=3   # average drop over first K comps
)
print(imp_loo.head(20))          # top contributors


## structure coefficients

In [ ]:
# Held-out structure coefficients: corr(feature_j, X1 canonical comp_k) on test folds
sc_df = cv_structure_coefficients(
    X2_sc, X1_sc, X2.columns,
    n_components=10, reg=1e-2,
    n_splits=5, random_state=0, groups=pn.y_var['segment'].values,
)
# Features most aligned with the first canonical mode:
print(sc_df.loc[:, ["feature", "SC_comp1"]].reindex(
      sc_df.index,).head(20))


# CCA (original class)

https://medium.com/@pozdrawiamzuzanna/canonical-correlation-analysis-simple-explanation-and-python-example-a5b8e97648d2

## Fit

In [ ]:
X2.shape

In [ ]:
# If error occurs, try reducing y_var again, and can also try pn.y_var_reduced.corr()[pn.y_var_reduced.corr() > 0.9]


X1 = pn.x_var_reduced[[col for col in pn.x_var_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
# X2 = pn.y_var_reduced.drop(columns=['bin', 'segment'], errors='ignore').copy()
X2 = pn.y_var.copy()
cca_no_lag = cca_class.CCAclass(X1=X1, 
                                X2=X2, 
                                lagging_included=False)
cca_no_lag.conduct_cca()

X1_lags = pn.x_var_lags_reduced[[col for col in pn.x_var_lags_reduced.columns if col.startswith('cluster_')]].drop(columns=['bin', 'segment'], errors='ignore').copy()
X2_lags = pn.y_var_lags_reduced.copy()
cca_lags = cca_class.CCAclass(X1=X1_lags, 
                              X2=X2_lags, 
                              lagging_included=True)
# for all columns that end with _0, rename them to the column name without the _0
cca_lags.X2.columns = cca_lags.X2.columns.str.replace('_0', '')
cca_lags.conduct_cca()


print(f'pn.x_var_lags.shape: {pn.x_var_lags.shape}')
print(f'pn.y_var_lags_reduced.shape: {pn.y_var_lags_reduced.shape}')

cca_inst = cca_lags

## X2 loadings

In [ ]:
cca_no_lag.plot_X2_loadings()

In [ ]:
cca_lags.plot_X2_loadings()

In [ ]:
stop!

# Test lags

## compared to baseline model

BASE = “baseline design matrix” (all other predictors, no lags for this feature).

BASE + best1 = baseline + the single best lag.

BASE + all = baseline + the whole lag block.

In [ ]:
# multiff_code/methods/neural_data_analysis/neural_analysis_tools/cca_methods/advanced_cca_tools/test_lags_in_cca.py
from neural_data_analysis.neural_analysis_tools.cca_methods.advanced_cca_tools import test_lags_in_cca, partial_cca_test, partition_x_utils

In [ ]:
features_to_test = ['monkey_speeddummy', 'RDz', 'gaze_mky_vew_y_l', 'gaze_mky_vew_y_r'
                    'abs_diff_in_angle_to_nxt_ff', 'abs_diff_in_abs_angle_to_nxt_ff',
                    'cur_cntr_arc_curv', 'turning_right']

for feature in features_to_test:
    # find lagged columns in X2_lags
    lag_cols = [col for col in X2_lags.columns if feature in col]
    if len(lag_cols) == 0:
        print(f"No lagged columns found for {feature}")
        continue
    # find their indices in X2_lags
    lag_cols_indices = [X2_lags.columns.get_loc(col) for col in lag_cols]


    print('===============================================')
    print(f'Testing {feature}')
    res = test_lags_in_cca.evaluate_lag_block_in_full_model(X2_lags, X1_lags, lag_block_cols=lag_cols_indices, n_splits=5, random_state=1)
    print("Test cancorr means:")
    print("  BASE                :", res.test_mean_BASE)
    print("  BASE + best single  :", res.test_mean_BASE_plus_best1)
    print("  BASE + all lags     :", res.test_mean_BASE_plus_all)
    print("Δ(all - BASE):", res.delta_all_vs_base, "  Δ(all - best1):", res.delta_all_vs_best1)
    print("Best single lag (per fold):", res.best_single_indices)
    print("Partial structure loadings (full model):", res.partial_loadings_full)

In [ ]:

# X_full: (n, p) design matrix with ALL features (including the lagged ones)
# Y:      (n, q) opposite view
# lag_block_cols: list of column indices in X_full for the ONE feature's lags you want to inspect
#
res = test_lags_in_cca.evaluate_lag_block_in_full_model(X2_lags, X1_lags, lag_block_cols=lag_cols_indices, n_splits=5, random_state=1)
print("Test cancorr means:")
print("  BASE                :", res.test_mean_BASE)
print("  BASE + best single  :", res.test_mean_BASE_plus_best1)
print("  BASE + all lags     :", res.test_mean_BASE_plus_all)
print("Δ(all - BASE):", res.delta_all_vs_base, "  Δ(all - best1):", res.delta_all_vs_best1)
print("Best single lag (per fold):", res.best_single_indices)
print("Partial structure loadings (full model):", res.partial_loadings_full)


## plot lag block loadings

In [ ]:
res = evaluate_lag_block_in_full_model(
    X_full=pn.y_var_lags_reduced,
    Y=pn.x_var_lags_reduced,
    lag_block_cols=lag_cols_indices,
    n_splits=5, random_state=1
)

plot_lag_block_loadings(res.partial_loadings_full,
                        best_single_indices=res.best_single_indices,
                        title="MyFeature lag footprint")

## Partial CCA incremental test

In [ ]:
# e.g., columns like: 'speed_lag0', 'speed_lag1', ..., 'speed_lag10'
out = partition_x_utils.partition_X_base_block(
    X=X2_lags, 
    block_name_regex=r"^monkey_speeddummy_\d+$"
)


In [ ]:
out.keys()

In [ ]:
res = partial_cca_test.partial_cca_incremental_cv(
    X2_lags, X1_lags, lag_block_cols=lag_cols_indices,
    n_splits=5,
    random_state=1
)

print("Mean TEST partial cancorr:", res.mean_test_partial_cancorr)
print("Fold TEST partial cancorr:", res.fold_test_partial_cancorr)
print("Mean TRAIN partial cancorr:", res.mean_train_partial_cancorr)
print("Notes:", res.notes)


## replace with bases

In [ ]:
from neural_data_analysis.neural_analysis_tools.glm_tools.tpg import glm_bases

# Suppose lag_block_cols = indices of this feature’s lags in X_full
X_full = X2_lags
lag_block_cols = lag_cols_indices

X_block = X_full.iloc[:, lag_block_cols].to_numpy()   # shape (n, L)
B, centers = glm_bases.raised_cosine_basis(n_basis=4, n_lags=X_block.shape[1])
X_basis = X_block @ B    # shape (n, 4)

mask_block = np.zeros(X_full[1], dtype=bool)
mask_block[np.asarray(lag_block_cols, dtype=int)] = True
    
# Now build a new X_full with raw lag block replaced
X_new = np.concatenate([X_full.iloc[:, ~mask_block].to_numpy(), X_basis], axis=1)
